In [1]:
import os
import random
from collections import defaultdict

import numpy as np
import pandas as pd
import nibabel
import matplotlib.style
matplotlib.style.use('ggplot')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
# import seaborn as sns
import matplotlib
%matplotlib inline
import numpy as np
from nilearn import datasets
from nilearn import plotting
import nibabel as nib

from neurolang import frontend as fe

/home/amachlou/miniconda3/envs/antonia/lib/python3.6/importlib/_bootstrap.py:205: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/amachlou/git/neurolang/neurolang/expression_pattern_matching.py:54: UserWarning: Warning in class FirstOrderLogicSolver overwrites {'universal_predicate', 'existential_predicate'} from base GenericSolver[typing.Any]
  warn(warn_message)
/home/amachlou/miniconda3/envs/antonia/lib/python3.6/importlib/_bootstrap_external.py:426: ImportWarning: Not importing directory /home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/mpl_toolkits: missing __init__
  _warnings.warn(msg.format(portions[0]), ImportWarning)


In [2]:
destrieux_dataset = datasets.fetch_atlas_destrieux_2009()
destrieux_map = nib.load(destrieux_dataset['maps'])
destrieux_dataset['labels'];

/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/numpy/lib/npyio.py:2278: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  output = genfromtxt(fname, **kwargs)


In [3]:
subjects = pd.DataFrame({
    'id': ['101309', '108121', '102008', '102311', '107321', '121315'],
    'gender': ['M', 'F', 'M', 'F', 'F', 'M'],
})

data_path = '/home/amachlou/git/neurolang/examples/Probability_maps/'

qs = range(1, 100, 2)

renaming_map = {
    'Marginal_sulcus_Cingulate': 'Marginal_sulcus',
    'Lateral_fissure_Lateral': 'Lateral_fissure',
    'Middle_frontal': 'Middle_frontal_sulcus',
    'Marginal_sulcus_Marginal': 'Marginal_sulcus',
    'Inferior_occipital': 'Inferior_occipital_sulcus',
    'Olfactory': 'Olfactory_sulcus',
    'Intralimbic': 'Intralimbic_sulcus',
    'Olfactory_sulcus_Olfactory': 'Olfactory_sulcus',
    'Inferior_frontal_sulcus_Inferior_frontal': 'Inferior_frontal_sulcus',
    'Orbital_H_shaped_sulcus_Orbital_H-shaped': 'Orbital_H_shaped_sulcus',
    'Inferior_temporal': 'Inferior_temporal_sulcus',
    'Intralimbic_sulcus_Cingulate': 'Intralimbic_sulcus',
    'Precentral': 'Precentral_sulcus',
    'Frontomarginal': 'Frontomarginal_sulcus',
    'Subparietal': 'Subparietal_sulcus',
    'Anterior_horizontal_ramus_lateral_fissure_Lateral': (
        'Anterior_horizontal_ramus_lateral_fissure'
    ),
    'Rhinal': 'Rhinal_sulcus',
    'Anterior_occipital': 'Anterior_occipital_sulcus',
    'Callosal_sulcus_Callosal': 'Callosal_sulcus',
    'Postcentral': 'Postcentral_sulcus',
    'Superior_frontal_sulcus_Superior_frontal': 'Superior_frontal_sulcus',
    'Lunate': 'Lunate_sulcus',
    'Occipitotemporal_sulcus_Occipitotemporal': (
        'Occipitotemporal_sulcus'
    ),
    'Intraparietal_sulcus_Intraparietal': 'Intraparietal_sulcus',
    'Inferior_frontal': 'Inferior_frontal_sulcus',
    'Angular_sulcus_Superior_temporal': 'Angular_sulcus',
    'Parieto_occipital_sulcus_Parieto-Occipital': 'Parieto_occipital_sulcus',
    'Superior_parietal_sulcus_Superior_parietal': 'Superior_parietal_sulcus',
'Inferior_temporal_sulcus_Inferior_temporal': 'Inferior_temporal_sulcus',
    'Cingulate': 'Cingulate_sulcus',
    'Retrocalcarine': 'Retrocalcarine_sulcus',
    'Superior_rostral': 'Superior_rostral_sulcus',
    'Paracentral_sulcus_Paracentral': 'Paracentral_sulcus',
    'Intermediate_primus_of_Jensen_Intermediate_primus_of': (
        'Intermediate_primus_of_Jensen'
    ),
    'Superior_frontal': 'Superior_frontal_sulcus',
    'Superior_rostral_sulcus_Superior_rostral': 'Superior_rostral_sulcus',
    'Superior_temporal_sulcus_Superior_temporal': 'Superior_temporal_sulcus',
    'Superior_occipital_sulcus_Superior_occipital': 'Superior_occipital_sulcus',
    'Intralingual': 'Intralingual_sulcus',
    'Lunate_sulcus_Lunate': 'Lunate_sulcus',
    'Superior_temporal': 'Superior_temporal_sulcus',
    'Hippocampal_sulcus_Hippocampal': 'Hippocampal_sulcus',
    'Collateral': 'Collateral_sulcus',
    'Superior_parietal': 'Superior_parietal_sulcus',
    'Cuneal_sulcus_Cuneal': 'Cuneal_sulcus',
    'Orbital_H_shaped_sulcus_Orbital': 'Orbital_H_shaped_sulcus',
    'Subparietal_sulcus_Subparietal': 'Subparietal_sulcus',
    'Anterior_vertical_ramus_lateral_fissure_Lateral': (
        'Anterior_vertical_ramus_lateral_fissure'
    ),
    'Superior_occipital': 'Superior_occipital_sulcus',
    'Cuneal': 'Cuneal_sulcus',
    'Temporopolar': 'Temporopolar_sulcus',
}

delete_sulci = {
    'Posterior_subcentral_sulcus',
    'Anterior_parolfactory_sulcus',
}


def clean_name(name):
    splits = name.split('_')
    count = len([s for s in splits if s == 'sulcus'])
    if count == 0:
        return name + '_sulcus'
    if count == 1:
        if name.endswith('_sulcus'):
            return name
        else:
            return name.split('_sulcus')[0] + '_sulcus'
        return name
    else:
        raise Exception('what')


manual_sulci = []
for _, subject in subjects.iterrows():
    for hemisphere in ('L', 'R'):
        surface_path = os.path.join(
            data_path,
            '{}.{}.pial.32k_fs_LR.surf.gii'.format(subject['id'], hemisphere)
        )
        surface = nibabel.load(surface_path)
        vertices = surface.darrays[0].data
        manual_sulci_gii_path = os.path.join(
            data_path, '{}.{}H.manual_drawings.func.gii'.format(
                subject['id'], hemisphere
            )
        )
        manual_sulci_gii = nibabel.load(manual_sulci_gii_path)
        for darray in manual_sulci_gii.darrays:
            sulcus_name = darray.meta.metadata['Name']
            sulcus_name = sulcus_name[:sulcus_name.rfind('_')]
            if sulcus_name in renaming_map:
                sulcus_name = renaming_map[sulcus_name]
            else:
                print(sulcus_name)
            if sulcus_name in delete_sulci: continue
            # sulcus_name = clean_name(sulcus_name)
            manual_sulci.append({
                'id': subject['id'],
                'gender': subject['gender'],
                'sulcus': sulcus_name,
                'hemisphere': hemisphere,
                'data': vertices[darray.data.nonzero()],
            })

/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/nibabel/gifti/parse_gifti_fast.py:62: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  newarr = np.fromstring(zdec, dtype=dt)
/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/nibabel/gifti/gifti.py:760: ResourceWarning: unclosed file <_io.BufferedReader name='/home/amachlou/git/neurolang/examples/Probability_maps/101309.L.pial.32k_fs_LR.surf.gii'>
  parser.parse(fptr=file_map['image'].get_prepare_fileobj('rb'))
/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/nibabel/gifti/gifti.py:760: ResourceWarning: unclosed file <_io.BufferedReader name='/home/amachlou/git/neurolang/examples/Probability_maps/101309.LH.manual_drawings.func.gii'>
  parser.parse(fptr=file_map['image'].get_prepare_fileobj('rb'))
/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/nibabel/gifti/gifti.py:760: ResourceW

Anterior_horizontal_ramus_lateral_fissure
Anterior_vertical_ramus_lateral_fissure
Angular_sulcus
Marginal_sulcus
Superior_temporal_sulcus
Superior_rostral_sulcus
Superior_parietal_sulcus
Superior_occipital_sulcus
Superior_frontal_sulcus
Subparietal_sulcus
Rhinal_sulcus
Precentral_sulcus
Postcentral_sulcus
Parieto_occipital_sulcus
Paracingulate_sulcus
Paracentral_sulcus
Orbital_H_shaped_sulcus
Olfactory_sulcus
Occipitotemporal_sulcus
Lateral_occipital_sulcus
Lateral_fissure
Intraparietal_sulcus
Intralingual_sulcus
Intermediate_primus_of_Jensen
Inferior_temporal_sulcus
Inferior_occipital_sulcus
Inferior_frontal_sulcus
Hippocampal_sulcus
Frontomarginal_sulcus
Cuneal_sulcus
Collateral_sulcus
Cingulate_sulcus
Central_sulcus
Callosal_sulcus
Calcarine_sulcus
Anterior_occipital_sulcus
Anterior_horizontal_ramus_lateral_fissure
Anterior_vertical_ramus_lateral_fissure
Angular_sulcus
Intermediate_primus_of_Jensen
Marginal_sulcus
Orbital_H_shaped_sulcus
Olfactory_sulcus
Superior_rostral_sulcus
Supe

/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/nibabel/gifti/gifti.py:760: ResourceWarning: unclosed file <_io.BufferedReader name='/home/amachlou/git/neurolang/examples/Probability_maps/102008.L.pial.32k_fs_LR.surf.gii'>
  parser.parse(fptr=file_map['image'].get_prepare_fileobj('rb'))
/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/nibabel/gifti/gifti.py:760: ResourceWarning: unclosed file <_io.BufferedReader name='/home/amachlou/git/neurolang/examples/Probability_maps/102008.LH.manual_drawings.func.gii'>
  parser.parse(fptr=file_map['image'].get_prepare_fileobj('rb'))
/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/nibabel/gifti/gifti.py:760: ResourceWarning: unclosed file <_io.BufferedReader name='/home/amachlou/git/neurolang/examples/Probability_maps/102008.R.pial.32k_fs_LR.surf.gii'>
  parser.parse(fptr=file_map['image'].get_prepare_fileobj('rb'))
/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/nibabel/gi

Superior_rostral_sulcus
Paracentral_sulcus
Anterior_horizontal_ramus_lateral_fissure
Anterior_vertical_ramus_lateral_fissure
Hippocampal_sulcus
Temporopolar_sulcus
Superior_temporal_sulcus
Superior_parietal_sulcus
Superior_occipital_sulcus
Superior_frontal_sulcus
Subparietal_sulcus
Retrocalcarine_sulcus
Precentral_sulcus
Posterior_subcentral_sulcus
Postcentral_sulcus
Parieto_occipital_sulcus
Paracingulate_sulcus
Orbital_H_shaped_sulcus
Olfactory_sulcus
Occipitotemporal_sulcus
Marginal_sulcus
Lateral_occipital_sulcus
Lateral_fissure
Intraparietal_sulcus
Intralingual_sulcus
Intermediate_primus_of_Jensen
Inferior_temporal_sulcus
Inferior_occipital_sulcus
Inferior_frontal_sulcus
Frontomarginal_sulcus
Cuneal_sulcus
Collateral_sulcus
Cingulate_sulcus
Central_sulcus
Callosal_sulcus
Calcarine_sulcus
Anterior_occipital_sulcus
Angular_sulcus
Anterior_horizontal_ramus_lateral_fissure
Anterior_vertical_ramus_lateral_fissure
Marginal_sulcus
Angular_sulcus
Hippocampal_sulcus
Superior_temporal_sulcus

/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/nibabel/gifti/gifti.py:760: ResourceWarning: unclosed file <_io.BufferedReader name='/home/amachlou/git/neurolang/examples/Probability_maps/107321.L.pial.32k_fs_LR.surf.gii'>
  parser.parse(fptr=file_map['image'].get_prepare_fileobj('rb'))
/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/nibabel/gifti/gifti.py:760: ResourceWarning: unclosed file <_io.BufferedReader name='/home/amachlou/git/neurolang/examples/Probability_maps/107321.LH.manual_drawings.func.gii'>
  parser.parse(fptr=file_map['image'].get_prepare_fileobj('rb'))
/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/nibabel/gifti/gifti.py:760: ResourceWarning: unclosed file <_io.BufferedReader name='/home/amachlou/git/neurolang/examples/Probability_maps/107321.R.pial.32k_fs_LR.surf.gii'>
  parser.parse(fptr=file_map['image'].get_prepare_fileobj('rb'))
/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/nibabel/gi

Angular_sulcus
Superior_occipital_sulcus
Intraparietal_sulcus
Inferior_occipital_sulcus
Anterior_vertical_ramus_lateral_fissure
Anterior_horizontal_ramus_lateral_fissure
Frontomarginal_sulcus
Temporopolar_sulcus
Paracingulate_sulcus
Superior_temporal_sulcus
Superior_rostral_sulcus
Superior_parietal_sulcus
Superior_frontal_sulcus
Subparietal_sulcus
Rhinal_sulcus
Precentral_sulcus
Postcentral_sulcus
Parieto_occipital_sulcus
Paracentral_sulcus
Orbital_H_shaped_sulcus
Olfactory_sulcus
Occipitotemporal_sulcus
Marginal_sulcus
Intralingual_sulcus
Lateral_occipital_sulcus
Inferior_rostral_sulcus
Collateral_sulcus
Inferior_temporal_sulcus
Intermediate_primus_of_Jensen
Cuneal_sulcus
Cingulate_sulcus
Middle_frontal_sulcus
Inferior_frontal_sulcus
Hippocampal_sulcus
Callosal_sulcus
Calcarine_sulcus
Lateral_fissure
Central_sulcus
Lunate_sulcus
Anterior_horizontal_ramus_lateral_fissure
Anterior_vertical_ramus_lateral_fissure
Marginal_sulcus
Intralimbic_sulcus
Superior_temporal_sulcus
Superior_rostral

/home/amachlou/miniconda3/envs/antonia/lib/python3.6/site-packages/nibabel/gifti/gifti.py:760: ResourceWarning: unclosed file <_io.BufferedReader name='/home/amachlou/git/neurolang/examples/Probability_maps/121315.RH.manual_drawings.func.gii'>
  parser.parse(fptr=file_map['image'].get_prepare_fileobj('rb'))


In [4]:
def spec_of_sulcus(manual_name, query_result_for_sulcus):
    '''
    Input: string of manually labelled sulcus, result from query for sulcus
    in the form of dictionary with subject id, hemisphere and list of found sulci for 5 random subjects out of 6
    Returns subject ids for which the query returned the desired sulcus only.
    '''
    print('Subjects where '+manual_name+' found only:')
    for x in range(0, 5):
        if manual_name in query_result_for_sulcus[x]['found_sulci'] and len(query_result_for_sulcus[x]['found_sulci'])==1:
            print(query_result_for_sulcus[x]['subject_id'])
#         elif len(query_result_for_sulcus[x]['found_sulci'])==0:
#             print(query_result_for_sulcus[x]['subject_id'] + ' empty')

### Superior rostral sulcus

In [5]:
results_srs = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #import pdb; pdb.set_trace()
        #SRS
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anterior_of(x, nl.symbols.manual_Callosal_sulcus) &
                nl.symbols.inferior_of(x, nl.symbols.manual_Callosal_sulcus) &
                ~nl.symbols.superior_of(x, nl.symbols.manual_Callosal_sulcus) &
                nl.symbols.isin(x, medial_sulci) &
                ~nl.symbols.isin(x, primary_sulci)

            )
        )
        results_srs.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
        
srs_spec = spec_of_sulcus('manual_Superior_rostral_sulcus', results_srs)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Superior_rostral_sulcus found only:
121315
102311
101309


In [6]:
results_srs

[{'subject_id': '107321',
  'hemisphere': 'R',
  'found_sulci': ['manual_Inferior_rostral_sulcus',
   'manual_Superior_rostral_sulcus']},
 {'subject_id': '121315',
  'hemisphere': 'R',
  'found_sulci': ['manual_Superior_rostral_sulcus']},
 {'subject_id': '102311',
  'hemisphere': 'R',
  'found_sulci': ['manual_Superior_rostral_sulcus']},
 {'subject_id': '108121',
  'hemisphere': 'R',
  'found_sulci': ['manual_Inferior_rostral_sulcus',
   'manual_Superior_rostral_sulcus']},
 {'subject_id': '101309',
  'hemisphere': 'R',
  'found_sulci': ['manual_Superior_rostral_sulcus']}]

In [7]:
target_sulcus='manual_Superior_rostral_sulcus'

for x in range(0, 5):
    if target_sulcus in results_srs[x]['found_sulci'] and len(results_srs[x]['found_sulci'])==1:
        print(results_srs[x]['subject_id'])

121315
102311
101309


In [8]:
all_found_sulci = set()
for res in results_srs:
    all_found_sulci = all_found_sulci.union(res['found_sulci'])
new_results = []
for result in results_srs:
    new_results.append({**{
        'subject_id': result['subject_id'],
        'hemisphere': result['hemisphere'],
    }, **{
        sulcus: sulcus in result['found_sulci'] for sulcus in all_found_sulci
    }})

In [9]:
import pandas as pd
df = pd.DataFrame(new_results)

In [10]:
df


,hemisphere,manual_Inferior_rostral_sulcus,manual_Superior_rostral_sulcus,subject_id
0,R,True,True,107321
1,R,False,True,121315
2,R,False,True,102311
3,R,True,True,108121
4,R,False,True,101309


In [11]:
df.sum()

hemisphere                                                 RRRRR
manual_Inferior_rostral_sulcus                                 2
manual_Superior_rostral_sulcus                                 5
subject_id                        107321121315102311108121101309
dtype: object

In [12]:
ratio_of_found_sulci = df.manual_Superior_rostral_sulcus.sum() / len(df.subject_id)
ratio_of_found_sulci

1.0

### Cingulate sulcus

In [13]:
results_cing = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #import pdb; pdb.set_trace()
        #cingulate
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anterior_of(x, nl.symbols.manual_Callosal_sulcus) &
                nl.symbols.anterior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.inferior_of(x, nl.symbols.manual_Callosal_sulcus) &
                nl.symbols.inferior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.superior_of(x, nl.symbols.manual_Callosal_sulcus) &
                nl.symbols.superior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.isin(x, medial_sulci) &
                ~nl.symbols.isin(x, primary_sulci)

            )
        )
        results_cing.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_cing[a]['found_sulci'])         
cing_spec = spec_of_sulcus('manual_Cingulate_sulcus', results_cing)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Cingulate_sulcus found only:
102008
121315


### Precentral sulcus

In [14]:
results_prcs = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #import pdb; pdb.set_trace()
        #prcs
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.posterior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.anatomical_anterior_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                nl.symbols.superior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.inferior_of(x, nl.symbols.manual_Callosal_sulcus) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)
            )
        )
        results_prcs.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
        
prcs_spec = spec_of_sulcus('manual_Precentral_sulcus', results_prcs)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Precentral_sulcus found only:
108121
121315
107321


### Postcentral sulcus

In [15]:
results_pocs = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #import pdb; pdb.set_trace()
        #pocs
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_posterior_of(x, nl.symbols.manual_Precentral_sulcus) &
                nl.symbols.anatomical_anterior_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                nl.symbols.superior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)
            )
        )
        results_pocs.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_pocs[a]['found_sulci'])        
pocs_spec = spec_of_sulcus('manual_Postcentral_sulcus', results_pocs)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Postcentral_sulcus found only:
102311
101309


### Intraparietal sulcus

In [16]:
results_ips = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #import pdb; pdb.set_trace()
        #ips
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.posterior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.anatomical_superior_of(x, nl.symbols.manual_Lateral_fissure) &
                nl.symbols.anatomical_anterior_of(x, nl.symbols.manual_Calcarine_sulcus) &
                nl.symbols.left_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)        
            )
        )
        results_ips.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_ips[a]['found_sulci'])
ips_spec = spec_of_sulcus('manual_Intraparietal_sulcus', results_ips)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Intraparietal_sulcus found only:
101309
107321


### Paracingulate sulcus

In [17]:
results_paracing = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #import pdb; pdb.set_trace()
        #paracing
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_anterior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.superior_of(x, nl.symbols.manual_Callosal_sulcus) &
                nl.symbols.isin(x, medial_sulci) &
                ~nl.symbols.isin(x, primary_sulci)
            )
        )
        results_paracing.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_paracing[a]['found_sulci'])        
paracing_spec = spec_of_sulcus('manual_Paracingulate_sulcus', results_paracing)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Paracingulate_sulcus found only:
102008
101309


In [18]:
results_paracing2 = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #import pdb; pdb.set_trace()
        #paracing
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_anterior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.superior_of(x, nl.symbols.manual_Callosal_sulcus) &
                nl.symbols.anterior_of(x, nl.symbols.manual_Callosal_sulcus) &
                nl.symbols.isin(x, medial_sulci) &
                ~nl.symbols.isin(x, primary_sulci)
            )
        )
        results_paracing2.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_paracing2[a]['found_sulci'])        
paracing_spec2 = spec_of_sulcus('manual_Paracingulate_sulcus', results_paracing2)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Paracingulate_sulcus found only:
101309
121315
107321
102008


### Intermediate primus of Jensen

In [19]:
results_jens = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #import pdb; pdb.set_trace()
        #ipoj
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_posterior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.anatomical_anterior_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                nl.symbols.superior_of(x, nl.symbols.manual_Lateral_fissure) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)       
            )
        )
        results_jens.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_jens[a]['found_sulci'])
jensen_spec = spec_of_sulcus('manual_Intermediate_primus_of_Jensen', results_jens)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Intermediate_primus_of_Jensen found only:
108121


### Angular sulcus

In [20]:
results_ang = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #import pdb; pdb.set_trace()
        #ang
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_posterior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.posterior_of(x, nl.symbols.manual_Lateral_fissure) &
#                 nl.symbols.inferior_of(x, nl.symbols.manual_Lateral_fissure) &
                nl.symbols.anatomical_anterior_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                nl.symbols.anatomical_superior_of(x, nl.symbols.manual_Calcarine_sulcus) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)       
            )
        )
        results_ang.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_ang[a]['found_sulci'])           
ang_spec = spec_of_sulcus('manual_Angular_sulcus', results_ang)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Angular_sulcus found only:


### Inferior frontal sulcus

In [21]:
results_ifs = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #import pdb; pdb.set_trace()
        #ifs
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_anterior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.inferior_of(x, nl.symbols.manual_Superior_frontal_sulcus) &
                nl.symbols.superior_of(x, nl.symbols.manual_Lateral_fissure) &
                ~nl.symbols.right_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)

            )
        )
        results_ifs.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_ifs[a]['found_sulci'])        
ifs_spec = spec_of_sulcus('manual_Inferior_frontal_sulcus', results_ifs)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Inferior_frontal_sulcus found only:
107321
108121
102008


### Inferior occipital sulcus

In [22]:
results_ios = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #import pdb; pdb.set_trace()
        #ios
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_inferior_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                nl.symbols.anatomical_posterior_of(x, nl.symbols.manual_Lateral_fissure) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)        
            )
        )
        results_ios.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_ios[a]['found_sulci'])
ios_spec = spec_of_sulcus('manual_Inferior_occipital_sulcus', results_ios)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Inferior_occipital_sulcus found only:


### Superior temporal sulcus

In [23]:
results_sts = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #import pdb; pdb.set_trace()
        #sts
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anterior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.posterior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.inferior_of(x, nl.symbols.manual_Calcarine_sulcus) &
                nl.symbols.superior_of(x, nl.symbols.manual_Calcarine_sulcus) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)

            )
        )
        results_sts.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_sts[a]['found_sulci'])
sts_spec = spec_of_sulcus('manual_Superior_temporal_sulcus', results_sts)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Superior_temporal_sulcus found only:
121315
107321
102008


### Anterior occipital sulcus

In [24]:
results_aos = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #import pdb; pdb.set_trace()
        #aos
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.posterior_of(x, nl.symbols.manual_Inferior_temporal_sulcus) &
                nl.symbols.inferior_of(x, nl.symbols.manual_Inferior_temporal_sulcus) &
                nl.symbols.anterior_of(x, nl.symbols.manual_Calcarine_sulcus) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)

            )
        )
        results_aos.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_aos[a]['found_sulci'])        
aos_spec = spec_of_sulcus('manual_Anterior_occipital_sulcus', results_aos)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Anterior_occipital_sulcus found only:


### Subparietal sulcus

In [25]:
results_subp = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #import pdb; pdb.set_trace()
        #subp
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anterior_of(x, nl.symbols.manual_Marginal_sulcus) &
                nl.symbols.anatomical_superior_of(x, nl.symbols.manual_Calcarine_sulcus) &
                nl.symbols.inferior_of(x, nl.symbols.manual_Marginal_sulcus) &
                nl.symbols.posterior_of(x, nl.symbols.manual_Marginal_sulcus) &
                nl.symbols.isin(x, medial_sulci) &
                ~nl.symbols.isin(x, primary_sulci)        
            )
        )
        results_subp.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_subp[a]['found_sulci'])
subp_spec = spec_of_sulcus('manual_Subparietal_sulcus', results_subp)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Subparietal_sulcus found only:
102311
107321
108121
121315
101309


### Frontomarginal sulcus

In [26]:
results_fms = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #import pdb; pdb.set_trace()
        #fms
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_anterior_of(x, nl.symbols.manual_Anterior_horizontal_ramus_lateral_fissure) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)        
            )
        )
        results_fms.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_fms[a]['found_sulci'])
fms_spec = spec_of_sulcus('manual_Frontomarginal_sulcus', results_fms)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Frontomarginal_sulcus found only:
101309
121315
102008
102311
108121


### Olfactory sulcus

In [27]:
results_olf = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #olf
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_anterior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.inferior_of(x, nl.symbols.manual_Lateral_fissure) &
                nl.symbols.left_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                nl.symbols.isin(x, ventral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)        
            )
        )
        results_olf.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_olf[a]['found_sulci'])
olf_spec = spec_of_sulcus('manual_Olfactory_sulcus', results_olf)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Olfactory_sulcus found only:
108121
121315
107321
102008
101309


### Orbital H-shaped sulcus

In [28]:
results_orbh = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #orbh
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_anterior_of(x, nl.symbols.manual_Lateral_fissure) &
                nl.symbols.anatomical_inferior_of(x, nl.symbols.manual_Lateral_fissure) &
                nl.symbols.right_of(x, nl.symbols.manual_Olfactory_sulcus) &
                nl.symbols.isin(x, ventral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)        
            )
        )
        results_orbh.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_orbh[a]['found_sulci'])
orbh_spec = spec_of_sulcus('manual_Orbital_H_shaped_sulcus', results_orbh)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Orbital_H_shaped_sulcus found only:
121315
101309
107321
102008
102311


### Temporopolar sulcus

In [29]:
results_tps = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #tps
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.posterior_of(x, nl.symbols.manual_Olfactory_sulcus) &
                nl.symbols.anterior_of(x, nl.symbols.manual_Collateral_sulcus) &
                nl.symbols.anatomical_inferior_of(x, nl.symbols.manual_Lateral_fissure) &
                nl.symbols.right_of(x, nl.symbols.manual_Olfactory_sulcus) &
                nl.symbols.isin(x, ventral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)        
            )
        )
        results_tps.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_tps[a]['found_sulci'])        
tps_spec = spec_of_sulcus('manual_Temporopolar_sulcus', results_tps)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Temporopolar_sulcus found only:


### Occipitotemporal sulcus

In [30]:
results_ots = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #ots
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.inferior_of(x, nl.symbols.manual_Lateral_fissure) &
                nl.symbols.anatomical_posterior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.right_of(x, nl.symbols.manual_Calcarine_sulcus) &
                nl.symbols.anatomical_anterior_of(x, nl.symbols.manual_Calcarine_sulcus) &
                nl.symbols.isin(x, ventral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)        
            )
        )
        results_ots.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_ots[a]['found_sulci'])
        
ots_spec = spec_of_sulcus('manual_Occipitotemporal_sulcus', results_ots)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Occipitotemporal_sulcus found only:
121315


### Retrocalcarine sulcus

In [31]:
results_retrocalc = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #retrocalc
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_posterior_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                ~nl.symbols.anatomical_superior_of(x, nl.symbols.manual_Calcarine_sulcus) &
                nl.symbols.isin(x, medial_sulci) &
                ~nl.symbols.isin(x, primary_sulci)

            )
        )
        results_retrocalc.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_retrocalc[a]['found_sulci'])
retrocalc_spec = spec_of_sulcus('manual_Retrocalcarine_sulcus', results_retrocalc)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Retrocalcarine_sulcus found only:
102008
102311
121315


### Inferior temporal sulcus

In [32]:
results_its = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #its
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anterior_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                nl.symbols.anatomical_inferior_of(x, nl.symbols.manual_Lateral_fissure) &
                nl.symbols.posterior_of(x, nl.symbols.manual_Lateral_fissure) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)

            )
        )
        results_its.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_its[a]['found_sulci'])        
its_spec = spec_of_sulcus('manual_Inferior_temporal_sulcus', results_its)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Inferior_temporal_sulcus found only:
108121
121315


In [33]:
results_its2 = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #its
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_anterior_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                nl.symbols.anatomical_inferior_of(x, nl.symbols.manual_Calcarine_sulcus) &
                nl.symbols.inferior_of(x, nl.symbols.manual_Superior_temporal_sulcus) &
                nl.symbols.anatomical_posterior_of(x, nl.symbols.manual_Anterior_vertical_ramus_lateral_fissure) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)

            )
        )
        results_its2.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_its2[a]['found_sulci'])
its_spec2 = spec_of_sulcus('manual_Inferior_temporal_sulcus', results_its2)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Inferior_temporal_sulcus found only:
121315
108121
102008


### Superior frontal sulcus

In [34]:
results_sfs = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #sfs
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anterior_of(x, nl.symbols.manual_Anterior_vertical_ramus_lateral_fissure) &
                nl.symbols.superior_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)      
            )
        )
        results_sfs.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_sfs[a]['found_sulci'])
sfs_spec = spec_of_sulcus('manual_Superior_frontal_sulcus', results_sfs)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Superior_frontal_sulcus found only:
102311
121315
102008


In [35]:
results_sfs2 = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #sfs
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anterior_of(x, nl.symbols.manual_Precentral_sulcus) &
                nl.symbols.anatomical_superior_of(x, nl.symbols.manual_Anterior_horizontal_ramus_lateral_fissure) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)      
            )
        )
        results_sfs2.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_sfs2[a]['found_sulci'])
sfs_spec2 = spec_of_sulcus('manual_Superior_frontal_sulcus', results_sfs2)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Superior_frontal_sulcus found only:
107321
121315
102008


### Hippocampal sulcus

In [36]:
results_hipp = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #hipp
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.inferior_of(x, nl.symbols.manual_Callosal_sulcus) &
                nl.symbols.posterior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.anatomical_anterior_of(x, nl.symbols.manual_Calcarine_sulcus) &
                nl.symbols.left_of(x, nl.symbols.manual_Occipitotemporal_sulcus) &
                nl.symbols.isin(x, ventral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)
            )
        )
        results_hipp.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_hipp[a]['found_sulci'])          
hipp_spec = spec_of_sulcus('manual_Hippocampal_sulcus', results_hipp)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Hippocampal_sulcus found only:
108121
121315


### Superior parietal sulcus

In [37]:
results_sps = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #sps
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_superior_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                nl.symbols.anatomical_posterior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)
            )
        )
        results_sps.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_sps[a]['found_sulci'])          
sps_spec = spec_of_sulcus('manual_Superior_parietal_sulcus', results_sps)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Superior_parietal_sulcus found only:
121315
102311


### Intralingual sulcus

In [38]:
results_ils = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #ils
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.inferior_of(x, nl.symbols.manual_Calcarine_sulcus) &
                nl.symbols.posterior_of(x, nl.symbols.manual_Collateral_sulcus) &
                nl.symbols.anatomical_posterior_of(x, nl.symbols.manual_Callosal_sulcus) &
                nl.symbols.anterior_of(x, nl.symbols.manual_Collateral_sulcus) &
                nl.symbols.left_of(x, nl.symbols.manual_Collateral_sulcus) &
                nl.symbols.isin(x, medial_sulci) &
                ~nl.symbols.isin(x, primary_sulci)

            )
        )
        results_ils.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_ils[a]['found_sulci'])        
        
ils_spec = spec_of_sulcus('manual_Intralingual_sulcus', results_ils)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Intralingual_sulcus found only:
121315
108121


### Collateral sulcus

In [39]:
results_coll = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #coll
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_inferior_of(x, nl.symbols.manual_Callosal_sulcus) &
                nl.symbols.anatomical_posterior_of(x, nl.symbols.manual_Anterior_horizontal_ramus_lateral_fissure) &
                nl.symbols.anterior_of(x, nl.symbols.manual_Calcarine_sulcus) &
                nl.symbols.right_of(x, nl.symbols.manual_Occipitotemporal_sulcus) &
                nl.symbols.isin(x, ventral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)

            )
        )
        results_coll.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# for a in range(0,5):
#     print(results_coll[a]['found_sulci'])        
coll_spec = spec_of_sulcus('manual_Collateral_sulcus', results_coll)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Collateral_sulcus found only:


### Paracentral sulcus

In [40]:
results_paracs = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #paracs
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_superior_of(x, nl.symbols.manual_Callosal_sulcus) &
                nl.symbols.anterior_of(x, nl.symbols.manual_Marginal_sulcus) &
                nl.symbols.posterior_of(x, nl.symbols.manual_Precentral_sulcus) &
                nl.symbols.isin(x, medial_sulci) &
                ~nl.symbols.isin(x, primary_sulci)

            )
        )
        results_paracs.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })

paracs_spec = spec_of_sulcus('manual_Paracentral_sulcus', results_paracs)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Paracentral_sulcus found only:
121315
102008


### (Calloso)marginal sulcus

In [41]:
results_marg = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #marg
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_superior_of(x, nl.symbols.manual_Callosal_sulcus) &
                nl.symbols.anterior_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                nl.symbols.superior_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                ~nl.symbols.anterior_of(x, nl.symbols.manual_Central_sulcus) &
                nl.symbols.isin(x, medial_sulci) &
                ~nl.symbols.isin(x, primary_sulci)

            )
        )
        results_marg.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
# print(results_marg)
marg_spec = spec_of_sulcus('manual_Marginal_sulcus', results_marg)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Marginal_sulcus found only:
102311
107321
121315


### Superior occipital sulcus

In [42]:
results_sos = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #sos
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_posterior_of(x, nl.symbols.manual_Lateral_fissure) &
                nl.symbols.anatomical_superior_of(x, nl.symbols.manual_Callosal_sulcus) &
                nl.symbols.inferior_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                nl.symbols.posterior_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                nl.symbols.left_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)

            )
        )
        results_sos.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
        
sos_spec = spec_of_sulcus('manual_Superior_occipital_sulcus', results_sos)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Superior_occipital_sulcus found only:
101309


### Lateral occipital sulcus

In [43]:
results_los = []
subject_ids = set([m['id'] for m in manual_sulci])
for subject_id in random.sample(subject_ids, len(subject_ids) - 1):
    for hemisphere in ('R', ):
        sulci = {
            m['sulcus']: m['data'] for m in manual_sulci 
            if m['id'] == subject_id and m['hemisphere'] == hemisphere
        }
        nl = fe.RegionFrontend()
        for name, points in sulci.items():
            ijk_points = nib.affines.apply_affine(np.linalg.inv(destrieux_map.affine), points).astype(int)
            region = fe.ExplicitVBR(
                ijk_points,
                destrieux_map.affine,
                img_dim=destrieux_map.shape
            )
            nl.add_region(region, result_symbol_name=f'manual_{name}')
            Primary_Sulci = set()
        for manual_sulcus_name in nl.region_names:
            if ('Lateral_fissure' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Central_sulcus' in manual_sulcus_name
                or 'Calcarine_sulcus' in manual_sulcus_name
                or 'Callosal_sulcus' in manual_sulcus_name):
                Primary_Sulci.add(nl.symbols[manual_sulcus_name])
        primary_sulci = nl.add_region_set(Primary_Sulci, name='primary_sulci')   
        Medial_surface_sulci = set()
        Lateral_surface_sulci = set()
        Ventral_surface_sulci = set()

        for manual_sulcus_name in nl.region_names:
            if ('Callosal' in manual_sulcus_name
                or 'Parieto_occipital' in manual_sulcus_name
                or 'Cingulate' in manual_sulcus_name 
                or 'alcarine' in manual_sulcus_name 
                or 'Marginal' in manual_sulcus_name
                or 'Paracingulate' in manual_sulcus_name
                or 'erior_rostral' in manual_sulcus_name
                or 'Paracentral' in manual_sulcus_name
                or 'Subparietal' in manual_sulcus_name
                or 'Cuneal' in manual_sulcus_name
                or 'Intralingual' in manual_sulcus_name
                or 'Intralimbic' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Medial_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Central' in manual_sulcus_name
                or 'Lateral_fissure' in manual_sulcus_name
                or 'ramus_lateral_fissure' in manual_sulcus_name
                or 'Precentral' in manual_sulcus_name
                or 'Postcentral' in manual_sulcus_name 
                or 'Superior_frontal' in manual_sulcus_name 
                or 'Inferior_frontal' in manual_sulcus_name
                or 'Middle_frontal' in manual_sulcus_name
                or 'Frontomarginal' in manual_sulcus_name
                or 'subcentral' in manual_sulcus_name
                or 'Intraparietal' in manual_sulcus_name
                or 'Intermediate_primus' in manual_sulcus_name
                or 'Superior_parietal' in manual_sulcus_name
                or 'Superior_occipital' in manual_sulcus_name
                or 'Inferior_occipital' in manual_sulcus_name 
                or 'Lateral_occipital' in manual_sulcus_name 
                or 'Lunate' in manual_sulcus_name
                or 'Superior_temporal' in manual_sulcus_name
                or 'Inferior_temporal' in manual_sulcus_name
                or 'Anterior_occipital' in manual_sulcus_name
                or 'Angular' in manual_sulcus_name
                or 'Medial_frontal' in manual_sulcus_name):
                Lateral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            elif ('Occipitotemporal' in manual_sulcus_name
                or 'Collateral' in manual_sulcus_name
                or 'Rhinal' in manual_sulcus_name 
                or 'Parahippocampal' in manual_sulcus_name 
                or 'Hippocampal' in manual_sulcus_name
                or 'Olfactory' in manual_sulcus_name
                or 'rbital' in manual_sulcus_name
                or 'Temporopolar' in manual_sulcus_name):
                Ventral_surface_sulci.add(nl.symbols[manual_sulcus_name])
            else:
                print(manual_sulcus_name)
        lateral_sulci = nl.add_region_set(Lateral_surface_sulci, name='lateral_sulci')
        medial_sulci = nl.add_region_set(Medial_surface_sulci, name='medial_sulci')
        ventral_sulci = nl.add_region_set(Ventral_surface_sulci, name='ventral_sulci')
        #los
        x = nl.new_region_symbol('x')
        q = nl.query(
            x, 
            (
                nl.symbols.anatomical_posterior_of(x, nl.symbols.manual_Lateral_fissure) &
                nl.symbols.inferior_of(x, nl.symbols.manual_Superior_occipital_sulcus) &
                nl.symbols.superior_of(x, nl.symbols.manual_Calcarine_sulcus) &
                nl.symbols.left_of(x, nl.symbols.manual_Parieto_occipital_sulcus) &
                nl.symbols.isin(x, lateral_sulci) &
                ~nl.symbols.isin(x, primary_sulci)

            )
        )
        results_los.append({
            'subject_id': subject_id,
            'hemisphere': hemisphere,
            'found_sulci': [s.symbol_name for s in q.do()]
        })
        
los_spec = spec_of_sulcus('manual_Lateral_occipital_sulcus', results_los)

/home/amachlou/git/neurolang/neurolang/regions.py:218: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  np.all(self.voxels == other.voxels))


Subjects where manual_Lateral_occipital_sulcus found only:
102311
